# Mass balance and mass action equations for chemical equilibrium calculations

In [ ]:
from reaktoro import *
db = Database("supcrt98.xml")
editor = ChemicalEditor(db)
editor.addAqueousPhaseWithElements("H O C")
editor.addGaseousPhase(["H2O(g)", "CO2(g)"])

system = ChemicalSystem(editor)

problem = EquilibriumProblem(system)
T = 100 # celsius
P = 50.0 # bars
problem.setTemperature(T, "celsius")
problem.setPressure(P, "bar")
problem.add("H2O", 100, "mol")
problem.add("CO2", 2, "mol")

In [ ]:
state = equilibrate(problem)

In [ ]:
b = state.elementAmounts()
n = state.speciesAmounts()
print("b = ", b)
print("n = ", n)
A = system.formulaMatrix()
print("A =\n", A)

Residual evalaution:

In [ ]:
r = b - np.dot(A, n)
import numpy as np
r_norm = np.linalg.norm(r)
print("||r|| = ", r_norm)

How much of the CO2(g) is dissolved as CO2(aq)?

In [ ]:
print(f"CO2(aq) amount is {state.speciesAmount('CO2(aq)'):6.4e} mol")

How much of the HO2(l) is evaporate as HO2(g)?

In [ ]:
print(f"HO2(g) amount is {state.speciesAmount('H2O(g)'):6.4e} mol")

What is the amount of H+(aq)?

In [ ]:
print(f"H+ amount is {state.speciesAmount('H+'):6.4e} mol")

In [ ]:
n, m = A.shape
for i in range(n):
    for j in range(m):
        print(f"{A[i][j]:4.0f}", end=" ")
    print("\n")

Rank is the maximal number of linearly independent columns of A, and is iqual to the dimension of the vector space spanned by its rows.

In [ ]:
rank = np.linalg.matrix_rank(A)
print("Rank of A is", rank)

The matrix from the lectures:

In [ ]:
A_ = [[2, 1, 1, 0, 1, 0, 0, 2],
      [1, 0, 1, 3, 3, 2, 2, 1],
      [0, 0, 0, 1, 1, 1, 1, 0],
      [0, 1, -1, -2, -1, 0, 0, 0]]
print("Rank of A is", np.linalg.matrix_rank(A_))

Which phases exist in equilibrium state? 

In [ ]:
phases_names = [phase.name() for phase in system.phases()]
stability_indices = state.phaseStabilityIndices()
print("Phases  : Phase amounts : Stability Indices")
for name, si in zip(phases_names, stability_indices):
    print(f"{name:>7} : {state.phaseAmount(name):13.4f} : {si}")

**Note**: The stability index of a *stable phase* is zero (or very close to zero), negative for *under-saturated phase*, and positive for the *over-saturated phase*.

In [ ]:
molar_masses = [element.molarMass() for element in system.elements()]
element_names = [element.name() for element in system.elements()]
for name, molar_mass in zip(element_names, molar_masses):
    print(f"{name} : {molar_mass:6.4e} (kg/mol)")

In [ ]:
for name, amount, molar_mass in zip(element_names, b, molar_masses):
    print(f"{name} : {amount:6.2f} mols and {amount*molar_mass*1e3:8.2f} g")

What are the amounts of each chemical species 

In [ ]:
species_names = [specie.name() for specie in system.species()]
print(species_names)
n = state.speciesAmounts()
print(n)

In [ ]:
i = 0
for name, amount in zip(species_names, n):
    species_molar_mass = np.dot(A[:, i], molar_masses) * 1e3 # in g
    mass = amount * species_molar_mass
    print(f"{name:>10} : {amount:8.4f} mols and {mass:10.4f} g")
    i += 1

In [ ]:
chemical_properties = system.properties(T, P, n)
print("Chemical potentials of the species:")
for potential, species, index in zip(
    chemical_properties.chemicalPotentials().val,
    system.species(),
    list(range(1, system.numSpecies()+1))
):
    print(f"\u03BC_{index} ({species.name():>8}) = {potential:>12.4f} (J/mol)")

In [ ]:
print("Logarithms of activities of the species:")
for activity, species, index in zip(
    chemical_properties.lnActivities().val,
    system.species(),
    list(range(1, system.numSpecies()+1))
):
    print(f"ln (a_{index}) ({species.name():>10}) = {activity:6.4e}")

In [ ]:
mu_H2O = - 242.992 * 1e3 # J / mol
mu_Hplus = 0 # J / mol
mu_OHminus = - 155.559 * 1e3 # J / mol

nu_H2O = 1
nu_Hplus = 1
nu_OHminus = 1

R = 8.314 # J / (mol * K)
T = 100 + 273.15 # K

lnK = - 1 / (R * T) * (nu_Hplus * mu_Hplus
                       + nu_OHminus * mu_OHminus 
                       - nu_H2O * mu_H2O)
print("lnK = ", lnK)

Recalling that $log_{10} K = \frac{ln K}{ln 10}$, we obtain:

In [ ]:
ln10 = 2.30
logK = lnK / ln10
print("logK = ", logK)